# Real-Time Stock Market Analysis and Visualization Pipeline

## Fetch Stock Data using yfinance

In [1]:
# Import required libraries
import yfinance as yf
import datetime

# Function to fetch real-time stock data
def fetch_stock_data(stock_symbols):
    """
    Fetch real-time stock data for provided stock symbols.
    
    Args:
        stock_symbols (list): List of stock symbols to fetch data for.
        
    Returns:
        dict: Stock data dictionary with stock_symbol, price, volume, and timestamp.
    """
    stock_data = {}
    
    for symbol in stock_symbols:
        # Fetch stock data from yfinance
        stock = yf.Ticker(symbol)
        
        # Get real-time data
        data = stock.history(period="1d", interval="1m").tail(1)  # Latest minute data
        
        if not data.empty:
            stock_data[symbol] = {
                "price": data["Close"].iloc[-1],
                "volume": data["Volume"].iloc[-1],
                "timestamp": datetime.datetime.now()
            }
        else:
            stock_data[symbol] = {
                "price": None,
                "volume": None,
                "timestamp": datetime.datetime.now()
            }

    return stock_data


# Example usage
stock_symbols = ["TCS.NS", "WIPRO.NS", "TECHM.NS", "INFY.NS", "HCLTECH.NS"]
result = fetch_stock_data(stock_symbols)

# Print the fetched data
for symbol, data in result.items():
    print(f"Stock Symbol: {symbol}")
    print(f"Price: ₹{data['price']}")
    print(f"Volume: {data['volume']}")
    print(f"Timestamp: {data['timestamp']}")


Stock Symbol: TCS.NS
Price: ₹4437.7998046875
Volume: 2254
Timestamp: 2024-12-10 18:37:29.760447
Stock Symbol: WIPRO.NS
Price: ₹307.75
Volume: 60542
Timestamp: 2024-12-10 18:37:30.146374
Stock Symbol: TECHM.NS
Price: ₹1762.1500244140625
Volume: 2759
Timestamp: 2024-12-10 18:37:30.598843
Stock Symbol: INFY.NS
Price: ₹1948.550048828125
Volume: 11315
Timestamp: 2024-12-10 18:37:30.986220
Stock Symbol: HCLTECH.NS
Price: ₹1936.3499755859375
Volume: 3644
Timestamp: 2024-12-10 18:37:31.254212


## Connect to MySQL Database

In [2]:
# Import required libraries
import pymysql

# Database connection details
host = 'localhost'  
user = 'root'       
password = 'Abc@1234'  
database = 'stock_prices'  

# Establish connection to MySQL
def connect_to_mysql():
    try:
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        print("Connected to MySQL Database successfully!")
        return connection
    except Exception as e:
        print(f"Error: Unable to connect to MySQL Database - {e}")
        return None


# Test the connection
conn = connect_to_mysql()

# Close the connection if it's established
if conn:
    conn.close()


Connected to MySQL Database successfully!


## Fetch and Insert Data into Database

In [3]:
def fetch_stock_data(stock_symbols):
    """
    Fetch real-time stock data for provided stock symbols.
    """
    stock_data = {}
    
    for symbol in stock_symbols:
        stock = yf.Ticker(symbol)
        data = stock.history(period="1d", interval="1m").tail(1)
        
        if not data.empty:
            stock_data[symbol] = {
                "price": data["Close"].iloc[-1],
                "volume": int(data["Volume"].iloc[-1]),
                "timestamp": datetime.datetime.now()
            }
        else:
            stock_data[symbol] = {
                "price": None,
                "volume": None,
                "timestamp": datetime.datetime.now()
            }
    return stock_data


# Function to insert data into MySQL database
def insert_into_db(connection, data):
    """
    Inserts stock data into the MySQL database.
    Args:
        connection: pymysql database connection
        data: Stock data dictionary
    """
    try:
        with connection.cursor() as cursor:
            for symbol, details in data.items():
                # Insert stock data
                sql = """
                INSERT INTO stock_data (stock_symbol, price, volume, timestamp) 
                VALUES (%s, %s, %s, %s)
                """
                cursor.execute(sql, (symbol, details['price'], details['volume'], details['timestamp']))
        
        # Commit the transaction
        connection.commit()
        print("Data inserted successfully into database.")
    except Exception as e:
        print(f"Error inserting data into database: {e}")
    finally:
        connection.close()


# Main ETL Logic
def main():
    # Stock symbols
    stock_symbols = ["TCS.NS", "WIPRO.NS", "TECHM.NS", "INFY.NS", "HCLTECH.NS"]

    # Fetch stock data
    data = fetch_stock_data(stock_symbols)
    print("Fetched stock data:\n", data)

    # Connect to MySQL
    connection = connect_to_mysql()
    if connection:
        # Insert data into the database
        insert_into_db(connection, data)


# Run the ETL process
if __name__ == "__main__":
    main()


Fetched stock data:
 {'TCS.NS': {'price': np.float64(4437.7998046875), 'volume': 2254, 'timestamp': datetime.datetime(2024, 12, 10, 18, 37, 39, 73104)}, 'WIPRO.NS': {'price': np.float64(307.75), 'volume': 60542, 'timestamp': datetime.datetime(2024, 12, 10, 18, 37, 39, 178888)}, 'TECHM.NS': {'price': np.float64(1762.1500244140625), 'volume': 2759, 'timestamp': datetime.datetime(2024, 12, 10, 18, 37, 39, 309180)}, 'INFY.NS': {'price': np.float64(1948.550048828125), 'volume': 11315, 'timestamp': datetime.datetime(2024, 12, 10, 18, 37, 39, 421601)}, 'HCLTECH.NS': {'price': np.float64(1936.3499755859375), 'volume': 3644, 'timestamp': datetime.datetime(2024, 12, 10, 18, 37, 39, 543360)}}
Connected to MySQL Database successfully!
Data inserted successfully into database.


## Fetch Data from MySQL And Visualize 

In [6]:
# Import required libraries
import pymysql
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Function to connect to MySQL and fetch data
def fetch_data_from_db():
    """
    Fetch data from MySQL database for visualization purposes.
    Returns:
        DataFrame: Data fetched from MySQL in pandas DataFrame format.
    """
    try:
        connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        
        # Fetch data
        query = "SELECT * FROM stock_data"
        df = pd.read_sql(query, connection)
        
        print("Data fetched successfully from MySQL database!")
        print(df.head())  # Preview data
        connection.close()
        return df
        
    except Exception as e:
        print(f"Error fetching data from database: {e}")
        return None


# Function to create the interactive dashboard
def create_dashboard(df):
    """
    Creates an interactive stock visualization dashboard using Plotly.
    Args:
        df (DataFrame): DataFrame containing data fetched from MySQL database.
    """
    if df is not None and not df.empty:
        # Convert the timestamp to datetime
        df['timestamp'] = pd.to_datetime(df['timestamp'])

        # Create traces for each stock symbol
        fig = make_subplots(rows=1, cols=1)

        for symbol in df['stock_symbol'].unique():
            # Filter data for each stock symbol
            stock_df = df[df['stock_symbol'] == symbol]
            
            # Create line traces
            trace = go.Scatter(
                x=stock_df['timestamp'],
                y=stock_df['price'],
                mode='lines',
                name=symbol
            )
            fig.add_trace(trace)

        # Update layout for better presentation
        fig.update_layout(
            title="Interactive Stock Price Dashboard",
            xaxis_title="Timestamp",
            yaxis_title="Stock Price (₹)",
            template="plotly_dark",
        )

        # Show the plot
        fig.show()
    else:
        print("No data available to visualize!")


# Main execution
def main():
    # Fetch data from database
    df = fetch_data_from_db()
    
    # Create and display the interactive dashboard
    create_dashboard(df)


# Run the script
if __name__ == "__main__":
    main()

Data fetched successfully from MySQL database!
   id stock_symbol    price  volume           timestamp
0   1       TCS.NS  4437.80    2254 2024-12-10 17:01:51
1   2     WIPRO.NS   307.75   60542 2024-12-10 17:01:51
2   3     TECHM.NS  1762.15    2759 2024-12-10 17:01:51
3   4      INFY.NS  1948.55   11315 2024-12-10 17:01:52
4   5   HCLTECH.NS  1936.35    3644 2024-12-10 17:01:52
